In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from collections import Counter
from datetime import timedelta,datetime
from calendar import monthrange
import plotly.express as px
pj = os.path.join

# Loading the data

In [ ]:
# Folder with the unzipped data
folder = r"Data_Sec\2020Q1"

In [ ]:
sub = pd.read_csv(pj(folder,"sub.txt"),sep="\t",dtype={"cik":str})

In [ ]:
sub.shape

## Exploring the columns

In [ ]:
def data_frame_stats(df):
    cnt = df.count()
    max_cnt = cnt.max()
    unique = df.nunique()
    
    return pd.DataFrame({
        "notnulls":cnt,
        "notnulls%":cnt/max_cnt,
        "unique":unique,
        "unique%":unique/max_cnt
        
    })


data_frame_stats(sub)

In [ ]:
# Let's have a look on one of the rows. Because it won't fit our line, we will transpose it to column
sub.head(1).T

## Date columns
We see three date columns, let's transforma them into dates:

In [ ]:
sub["period"] = pd.to_datetime(sub["period"], format="%Y%m%d")
sub["filed"] = pd.to_datetime(sub["filed"], format="%Y%m%d")
sub["accepted"] = pd.to_datetime(sub["accepted"])

In [ ]:
# Let's check that they were correctly loaded
sub.select_dtypes("datetime").head()

# Sumbission forms (type of the reports)
The columns `"form"` contains information about the type of submissiong as described on the SEC page. https://www.sec.gov/forms. What are the most common submission forms?

In [ ]:
# distribution of the different types of reports based on the analysis of the form field
sub["form"].value_counts().plot(kind="bar")
plt.title("Which fillings are reported the most")
plt.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'pie'}]],)

# count different form types
df = sub["form"].value_counts().to_frame()
fig.add_trace(
    go.Bar(y=df["form"], x=df.index, text=df["form"], name="Forms - Bar Chart"),
    row=1, col=1,
)

# on the pie chart display only 5 most common categories and name the rest as "Other"
df["label"] = np.where(df["form"].rank(ascending=False)<6, df.index, "Other")
df = df.groupby("label").sum()

fig.add_trace(
    go.Pie(values=df["form"], labels=df.index,textinfo='label+percent'),
    row=1, col=2
)

fig.show()

We can see that the most common form is `8-K`. What does it contain and do we have some detailed data related to this submission type? 

## 8-K

In [ ]:
# let's filter adsh, unique report identifier of the `8-K`s
eight_k_filter = sub[sub["form"]=="8-K"][["name","adsh"]]
eight_k_filter.shape

In [ ]:
# load the num.txt file containing detailed data
num = pd.read_csv(pj(folder,"num.txt"),sep="\t")

In [ ]:
# merge the file headers with the detailed data 
eight_k_nums = num.merge(eight_k_filter)
eight_k_nums.shape

Out of 7502 8-K records, there are only 2894 rows in the num file which contain the data values. But is it 2894 adsh? 

In [ ]:
len(eight_k_nums["adsh"].unique())

No. From 7502,there are only 20 records in the dataset containing values. The rest is missing. Let's have a look what these values cover. 

In [ ]:
eight_k_nums.loc[0].T

On the url https://sec.report/Document/0001262976-20-000015/ you can see the details of this 8-K form. It's Item 8.01, other event, as desribec on the SEC page https://www.sec.gov/fast-answers/answersform8khtm.html. Other Events (The registrant can use this Item to report events that are not specifically called for by Form 8-K, that the registrant considers to be of importance to security holders.) This event can contain full or partial finance statement, like in this case of CIMPRESS company. The value "Accounts payable" appear on the page 5 of the report. 

# 10-K and 10-Q
## Do companies report anual report in Q1?
What interests us if not the most common form type? It should be the real financial statements. SEC explain the form types on this page: https://www.sec.gov/forms. When you study it a bit, you realize that we're concerned about: 
* `10-K` Anual report
* `10-Q` Quarterly report
and maybe
* `20-F` Anual Reports of a foreign company
* `40-F` Anual Reports of a foreign company

Let's first have a look on US companies submitting `10-K` and `10-Q`. We have loaded the data for `2020Q1` and we expect that every company reports just a single submission, most likely `10-K` anual report for the end of the year 2019. 

In [ ]:
tens = sub[sub["form"].isin(["10-Q","10-K"])]
tens_counts = tens["form"].value_counts().reset_index()
f1 = px.bar(tens_counts,y="form",x="index",barmode='group', title="Number of forms in 2020Q1")
f1.show()

## Does everyone report only one 10-K or 10-Q?

As expected most of the submisison in 2020Q1 are anual records for 2019. However there are some quarterly records. We will look at those later. Let's first check if each company reports only one set of financial statements. 

In [ ]:
# Is 10-Q reported once by each company or more times? 
sub[sub["form"].isin(["10-Q","10-K"])]["cik"].value_counts().value_counts().sort_index().plot(kind="bar")
plt.title("How many 10-Q reported each company")
plt.show()

No. Some companies (even though only a few) are uploading more than one statetement. What could that mean?

In [ ]:
df = sub[sub["form"].isin(["10-Q","10-K"])][["form","cik"]].groupby(["cik","form"]).size().to_frame('size').reset_index().groupby(["form","size"]).size().to_frame('count').reset_index()

f2 = px.bar(df[df["size"]>1], x="size", y="count",barmode="group", color="form", title="Number of companies having more than 1 submission in 2020Q1")
f2.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1
    )
)
f2.show()

In [ ]:
# attemps to make charts as subplots failed
#fig = make_subplots(rows=2, cols=1,subplot_titles=(f1["layout"].title.text, f2["layout"].title.text))
#fig.add_trace(f1['data'][0],row=1, col=1)
#fig.add_trace(f2['data'][0],row=2, col=1)
#fig.add_trace(f2['data'][1],row=2, col=1)
#fig

In [ ]:
# let's review which companies are having more than one submission by 
# exploring the ["cik"].value_counts a serie containing only count per each Central Index Key (cik)
companies_reporting_more_tens = tens["cik"].value_counts()[tens["cik"].value_counts()>1]
companies_reporting_more_tens.head()

In [ ]:
# note that results of the value_counts is a serie
type(tens["cik"].value_counts()), type(companies_reporting_more_tenq)

In [ ]:
# finally let's have a look on these companies. 
# there are too many column to see where they differ, but I would first check the dates
# only two columns have date format - period and filled
columns_to_see = ["adsh","cik","name","period","fy","fp","filed", "form"]
tens[tens["cik"].isin(companies_reporting_more_tens.index)].sort_values(by=["cik","period"])[columns_to_see]

We can really see that mostly the filings are representing a different period (Note: period column contains - balance sheet date rounded to the nearest end of the month). To be sure, let's groupby accros both the cik and the period

In [ ]:
# by applying max to the size, we would see if some company have more than one report in the same period
# the maximum is two which mean that some companies are reporting more than one 10-Q for the same period, let's explore them
tens.groupby(["cik","period"]).size().max()

Two companies have submited two reports in this quater, let's see how they differ?

In [ ]:
# what are the companies and the periods? 
duplicates = tens.groupby(["cik","period"]).size()[tens.groupby(["cik","period"]).size()>1]
duplicates

In [ ]:
# groupby().size() will results in a serie with multiindex, which can be merged on the original dataframe after the index is reset
duplicates = tens.merge(duplicates.reset_index(), on=["cik","period"]).sort_values(by=["cik","period"]).T
duplicates

So what are the differencesof those doubled lines? 

In [ ]:
# let's have a look at rows 2,3 which belongs to the company "CHUN CAN CAPITAL GROUP"
duplicates["same23"] = duplicates[2]==duplicates[3]
duplicates.loc[duplicates["same23"]==False,[3,2,"same23"]].dropna()

In [ ]:
# let's have a look at rows 2,3 which belongs to the company "ZOOMPASS HOLDINGS, INC.""
duplicates["same01"] = duplicates[0]==duplicates[1]
duplicates.loc[duplicates["same01"]==False,[0,1,"same01"]].dropna()

We see that they two records differe only in`filed` date and `accepted` time and adsh (which is unique for each records).The first example only differ in `accepted` Let's pick the latest filed and in case the filed is the same, the latest accepted data of the records.

### Filter out duplicates, only the latest updated recordb

In [ ]:
# the shape of our file containing `10-K` and `10-Q` is 5212 rows and 36 columns
tens.shape

Because it's theoretically possible that a file which was accepted later has lower filled, the only way how to filter out the duplicated lines is:
* sort data by the decisive fields - `filed` and `accepted`
* group by partiotioning columns - `cik` and `period`
* use `.cumcount()` to rank the rows 0,1,2 ...
* filter only row which have this value `==0`

Let's see an exmaple of the `cik` = `1191334`, we need to identify the index `3289` and filter out `3288`

In [ ]:
# example cik 1191334 and period 2013-09-30
tens[(tens["cik"]=="1191334")&(tens["period"]=="20130930")][["adsh","cik","period","filed","accepted"]].sort_values(by=["filed","accepted"],ascending=True)

In [ ]:
tens[(tens["cik"]=="1191334")&(tens["period"]=="20130930")][["adsh","cik","period","filed","accepted"]].sort_values(by=["filed","accepted"],ascending=False).groupby(["cik","period"]).cumcount()

In [ ]:
# for all the `10-K` and `10-Q` we reduce the number of rows from 5212 to 5210
tens.loc[tens.sort_values(by=["filed","accepted"],ascending=False).groupby(["cik","period"]).cumcount()==0]

## Summision dates
The 2020Q1 gets available in the beginning of April 2020. We would assume that it takes some time to process the data, so it won't probably contain financial statements from March, maybe not even Feb-2020. It could have data for Jan-2020, Dec-2019 and Nov-2019. What is the reality? 

In [ ]:
tens["period"].value_counts().reset_index().head()

In [ ]:
px.bar(tens["period"].value_counts().reset_index(), y="period", x="index")
fig.update_layout(yaxis_type="log")

Some of the companies have sent more than 1 submission. Let's filter only the latest period per `cik` to see what are the dates the companies have reported in 2020Q1. 

In [ ]:
tens_latest_period = tens.loc[tens.sort_values(by="period", ascending=False).groupby("cik").cumcount()==0]

In [ ]:
# using logarithim y-axis we can see the records in ranges of single-hits, tens, hundreds and thousands
fig= px.bar(tens_latest_period["period"].value_counts().reset_index(), y="period", x="index")
fig.update_layout(yaxis_type="log")

Our dataset `tens_latest_period` contains only the latest submission `period` for each `cik`

In [ ]:
# each cik contains maximum one submission in the set
tens_latest_period.groupby(["cik","period"]).size().max()

## Average processing delay
In this sesction we will review how much time elapse between the financial reporting period end and when the data are accepted into EDGAR system. The rules say that the companies should delivery quarterly report in 40-45 days and annual in 60-90 days depending on the size of the company (https://www.sec.gov/fast-answers/answers-form10khtm.html). But what is the reality.

In [ ]:
pd.options.mode.chained_assignment = None  # switch off the warnings
tens_latest_period["delivery_time"] = tens_latest_period["accepted"] - tens_latest_period["period"]
tens_latest_period["delivery_time"] = tens_latest_period["delivery_time"].dt.days

In [ ]:
u = tens_latest_period[["form","period","accepted","delivery_time"]]

In [ ]:
tens_latest_period["delivery_time"].agg(["mean","median","min","max"])

In [ ]:
px.histogram(tens_latest_period, x="delivery_time", color="form")

Most of the quarterly reports are delivered in 40 days and annual in sixty. Larger companies use their privilidge to extend the deadline to 90 days. In rare cases however the financial sheet arrives later.